# Purpose
This notebook tests the naive training of the the completion network.

# Import

In [1]:
import torch
import torchvision

from glic.networks.completion_network import CompletionNetwork
from glic.training.cn_training import train_cn

# Train

In [2]:
# load the dataset
train_dataset_dir = '../data/train/'
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_dir, transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=False)

In [3]:
from glic.utils import list_files

In [4]:
checkpoint = "amphitheater/00000005.jpg"

In [8]:
list_files('../data/train/').index(checkpoint)

25004

In [6]:
import os
def get_dataloader(dir: str, resume_path: str, batch_size):
    resume_index = os.listdir(train_dataset_dir).index(resume_path)
    # return dataloader
    pass

In [7]:
dataloader = get_dataloader(train_dataset_dir, "amphitheater/00000005.jpg", batch_size=)

SyntaxError: invalid syntax (875272085.py, line 1)

In [ ]:
# others
cn = CompletionNetwork()
optimizer = torch.optim.Adadelta(cn.parameters(), lr=2e-4)
num_batch = 10
replacement_val = torch.rand(3)

In [4]:
# train
_ = train_cn(cn, optimizer, train_dataloader, num_batch, replacement_val)


### BATCH 0 ###
loss: 2229.089599609375

### BATCH 1 ###
loss: 2465.78759765625

### BATCH 2 ###
loss: 2301.607666015625

### BATCH 3 ###
loss: 3246.95654296875

### BATCH 4 ###
loss: 2866.7548828125

### BATCH 5 ###
loss: 3330.05859375

### BATCH 6 ###
loss: 2269.8427734375

### BATCH 7 ###
loss: 2663.375244140625

### BATCH 8 ###
loss: 2629.3564453125

### BATCH 9 ###
loss: 3134.853515625
